In [1]:
import math
import pandas as pd
import numpy as np
import pbhs as pbhs

from scipy import constants
from astropy import constants as astro_const
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt



import black_holes as bh

solar_mass_grams = 1000 * astro_const.M_sun.value

# PBHs

## Quantum evaporation and absorption of energy

Black holes emit thermal radiation at a temperature inversely proportional to their mass. Numerically, this effect is given by

\begin{equation}
T_{bh}=\frac{\hbar{c}^{3}}{8\pi{k_{B}G}M}\simeq \frac{6.2\times{10}^{-8}K}{(M/M_{\odot})}
\end{equation}




In [2]:
solar_mass_units = np.arange(1, 101)
masses_kg = solar_mass_units * astro_const.M_sun.value

# Calculate the Hawking temperature for each mass
hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, astro_const.M_sun.value) for m in masses_kg]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_solar) from import': solar_mass_units,
    'Mass (kg)': masses_kg,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)

    Mass (M_solar) from import     Mass (kg)  Hawking Temperature (K)
0                            1  1.988410e+30             6.200000e-08
1                            2  3.976820e+30             3.100000e-08
2                            3  5.965230e+30             2.066667e-08
3                            4  7.953639e+30             1.550000e-08
4                            5  9.942049e+30             1.240000e-08
..                         ...           ...                      ...
95                          96  1.908873e+32             6.458333e-10
96                          97  1.928758e+32             6.391753e-10
97                          98  1.948642e+32             6.326531e-10
98                          99  1.968526e+32             6.262626e-10
99                         100  1.988410e+32             6.200000e-10

[100 rows x 3 columns]


In [3]:
temperature_critical_mass = pbhs.temperature_critical_mass(2.7) / solar_mass_grams
dmdt, first, second = pbhs.dm_dt(5 * solar_mass_grams, 2.7)

print(f"mass_c({temperature_critical_mass} K): {temperature_critical_mass:.2e}")
print(f"dmdt: {dmdt:.2e}")
print(f" - first: {first:.2e}")
print(f" - second: {second:.2e}")


mass_c(2.2910545872296856e-08 K): 2.29e-08
dmdt: 9.19e-11
 - first: -4.01e-44
 - second: 9.19e-11


In [10]:
background_temperature = 2.7
solar_mass_units = np.arange(1, 101)
masses_g = solar_mass_units * solar_mass_grams

temperature_critical_mass = pbhs.temperature_critical_mass(background_temperature) / solar_mass_grams

print(f"temperature_critical_mass({background_temperature} K): {temperature_critical_mass:.2e} grams")

# Calculate the Hawking temperature for each mass
terms_tuple = [pbhs.dm_dt(masses_g, background_temperature) for m in masses_g]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_solar)': solar_mass_units,
    'Mass (kg)': masses_g,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)

temperature_critical_mass(2.7 K): 2.29e-08 grams
    Mass (M_solar) from import     Mass (kg)  Hawking Temperature (K)
0                            1  1.988410e+30             6.200000e-08
1                            2  3.976820e+30             3.100000e-08
2                            3  5.965230e+30             2.066667e-08
3                            4  7.953639e+30             1.550000e-08
4                            5  9.942049e+30             1.240000e-08
..                         ...           ...                      ...
95                          96  1.908873e+32             6.458333e-10
96                          97  1.928758e+32             6.391753e-10
97                          98  1.948642e+32             6.326531e-10
98                          99  1.968526e+32             6.262626e-10
99                         100  1.988410e+32             6.200000e-10

[100 rows x 3 columns]
